# Used Car Market Analysis and inventory recommendation
This notebook analyzes used‑car listings for Los Angeles, Dallas, and Columbus. Recommends the inventry to stock.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid", context="notebook")
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 100)

In [ ]:
# Brand-specific color palette
brand_colors = {
    "Toyota": "#1f77b4", "Honda": "#ff7f0e", "Ford": "#2ca02c", "Chevrolet": "#d62728",
    "Nissan": "#9467bd", "Hyundai": "#8c564b", "Kia": "#e377c2", "Volkswagen": "#7f7f7f",
    "Subaru": "#bcbd22", "Mazda": "#17becf", "Jeep": "#1f77b4", "Dodge": "#ff7f0e",
    "Ram": "#2ca02c", "GMC": "#d62728", "Buick": "#9467bd", "Chrysler": "#8c564b",
    "Mitsubishi": "#e377c2", "BMW": "#7f7f7f", "Mercedes-Benz": "#bcbd22", "Audi": "#17becf",
    "Lexus": "#1f77b4", "Acura": "#ff7f0e", "Infiniti": "#2ca02c", "Volvo": "#d62728",
    "Lincoln": "#9467bd", "Cadillac": "#8c564b", "Land Rover": "#e377c2", "Jaguar": "#7f7f7f",
    "Porsche": "#bcbd22", "Alfa Romeo": "#17becf", "Genesis": "#1f77b4", "Mini": "#ff7f0e",
    "Fiat": "#2ca02c", "Tesla": "#d62728", "Rivian": "#9467bd", "Lucid": "#8c564b",
    "Polestar": "#e377c2", "Aston Martin": "#7f7f7f", "Maserati": "#bcbd22",
    "Bentley": "#17becf", "Rolls-Royce": "#1f77b4"
}

## Load Dataset
Load dataset and inspect its structure.

In [ ]:
DATA_PATH = "used_car_data.csv"
df = pd.read_csv(DATA_PATH)
print("Loaded dataset with shape:", df.shape)
df.head()

## Data Quality Report
Check for missing values and unrealistic values in dataset

In [ ]:
print("Missing values per column:\n")
print(df.isna().sum())

print("\nChecking unrealistic values...\n")
unrealistic = {}

unrealistic["negative_mileage"] = (df["mileage"] < 0).sum()
unrealistic["high_mileage"] = (df["mileage"] > 300000).sum()
unrealistic["zero_price"] = (df["price"] <= 0).sum()
unrealistic["high_price"] = (df["price"] > 250000).sum()
unrealistic["bad_year_low"] = (df["year"] < 1990).sum()
unrealistic["bad_year_high"] = (df["year"] > 2025).sum()
unrealistic["invalid_body_type"] = (df["body_type"] == "Unknown").sum()

pd.DataFrame.from_dict(unrealistic, orient="index", columns=["count"])

## Clean Data
Drop rows with missing required fields and unrealistic values.

In [ ]:
print("Initial shape:", df.shape)

required_cols = ["price", "year", "mileage", "make", "model", "market"]
df_clean = df.dropna(subset=required_cols).copy()
print("After dropping missing required fields:", df_clean.shape)

before = df_clean.shape[0]

df_clean = df_clean[
    (df_clean["price"] > 1000) & (df_clean["price"] < 250000) &
    (df_clean["year"] >= 1990) & (df_clean["year"] <= 2025) &
    (df_clean["mileage"] >= 0) & (df_clean["mileage"] <= 300000) &
    (df_clean["body_type"] != "Unknown")
]

after = df_clean.shape[0]
print(f"Rows removed due to unrealistic values: {before - after}")
print("Final cleaned shape:", df_clean.shape)
df_clean.head(100)

## Sanity Checks
Verify that the cleaned dataset is within expected ranges.

In [ ]:
print("Year range:", df_clean["year"].min(), "-", df_clean["year"].max())
print("Price range:", df_clean["price"].min(), "-", df_clean["price"].max())
print("Mileage range:", df_clean["mileage"].min(), "-", df_clean["mileage"].max())
print("Markets:", df_clean["market"].unique())

## Create Features
Compute car age, price‑per‑mile, and price segments for each market.

In [ ]:
CURRENT_YEAR = 2025
df_clean["age"] = CURRENT_YEAR - df_clean["year"]
df_clean["price_per_mile"] = df_clean["price"] / df_clean["mileage"].replace(0, np.nan)

df_clean.loc[df_clean["price_per_mile"] > 10, "price_per_mile"] = np.nan

for market, group in df_clean.groupby("market"):
    q = group["price"].quantile([0.33, 0.66])
    df_clean.loc[group.index, "price_segment"] = group["price"].apply(
        lambda x: "Low" if x <= q[0.33] else ("Mid" if x <= q[0.66] else "High")
    )

df_clean.head(100)

## Market Summary
Generate a high‑level overview of pricing, age, and mileage across markets.

In [ ]:
summary = df_clean.groupby("market").agg(
    n_listings=("price", "size"),
    median_price=("price", "median"),
    mean_price=("price", "mean"),
    median_age=("age", "median"),
    median_mileage=("mileage", "median")
).reset_index()
summary

## Price, Age, and Mileage Distributions
Compare key attribute distributions across markets.

In [ ]:
plt.figure(figsize=(14,4))

plt.subplot(1,3,1)
sns.boxplot(data=df_clean, x="market", y="price")
plt.title("Price Distribution")

plt.subplot(1,3,2)
sns.boxplot(data=df_clean, x="market", y="age")
plt.title("Age Distribution")

plt.subplot(1,3,3)
sns.boxplot(data=df_clean, x="market", y="mileage")
plt.title("Mileage Distribution")

plt.tight_layout()
plt.show()

## Top Makes
Identify the most frequently listed makes.

In [ ]:
markets = ["Los Angeles", "Dallas", "Columbus"]

for m in markets:
    subset = top_makes[top_makes["market"] == m].copy()

    subset["color"] = subset["make"].map(brand_colors)

    plt.figure(figsize=(8, 5))

    ax = sns.barplot(
        data=subset,
        y="make", x="count",
        orient="h",
        color="#CCCCCC"   # placeholder
    )

    for bar, color in zip(ax.patches, subset["color"]):
        bar.set_color(color)

    plt.title(f"Top Makes in {m}")
    plt.xlabel("Listing Count")
    plt.ylabel("Make")
    plt.tight_layout()
    plt.show()

## Inventory Recommendations
Score makes/models using frequency and price segment to highlight strong inventory candidates.

In [ ]:
def inventory_recommendations(df_market, top_n=10):
    seg_weights = {"Low": 1, "Mid": 2, "High": 3}
    tmp = df_market.copy()
    tmp["segment_weight"] = tmp["price_segment"].map(seg_weights)

    rec = (
        tmp.groupby(["make", "model"]).agg(
            n_listings=("price", "size"),
            avg_price=("price", "mean"), 
            median_age=("age", "median"),
            median_mileage=("mileage", "median"),
            segment_score=("segment_weight", "mean")
        ).reset_index()
    )

    rec["score"] = rec["n_listings"] * rec["segment_score"]
    return rec.sort_values("score", ascending=False).head(top_n)


for market in df_clean["market"].unique():
    recs = inventory_recommendations(df_clean[df_clean["market"] == market]).copy()

    print(f"\n=== Inventory Recommendations for {market} ===")
    display(recs)

    recs["label"] = recs["make"] + " " + recs["model"]

    recs["color"] = recs["make"].map(brand_colors)

    plt.figure(figsize=(9, 5))
    ax = sns.barplot(
        data=recs,
        y="label",
        x="score",
        orient="h",
        color="#CCCCCC"   # placeholder color
    )

    for bar, color in zip(ax.patches, recs["color"]):
        bar.set_color(color)

    plt.title(f"Top Inventory Recommendations in {market}")
    plt.xlabel("Recommendation Score")
    plt.ylabel("Make / Model")
    plt.tight_layout()
    plt.show()